In [ ]:
import os
import pandas as pd
from IPython.display import Markdown, HTML, display

In [1]:
import os

# Replace with your actual values
os.environ["AZURE_OPENAI_ENDPOINT"] = "INSERT THE OPENAI ENDPOINT"
os.environ["AZURE_OPENAI_API_KEY"] = "INSERT YOUR OPENAI API KEY"


In [ ]:
# 2nd Cell: Azure OpenAI setup
import os
from langchain_openai import AzureChatOpenAI
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

# Load your Azure environment variables
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_DEPLOYMENT_NAME = "gpt-4.1"  # 👈 Change if needed
AZURE_API_VERSION = "2025-01-01-preview"  # 👈 Use your correct version

# Define Azure LLM with streaming enabled
model = AzureChatOpenAI(
    openai_api_version=AZURE_API_VERSION,
    azure_deployment=AZURE_DEPLOYMENT_NAME,
    azure_endpoint=AZURE_OPENAI_ENDPOINT,
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)


In [4]:
# --- Setup ---
import os
import gradio as gr
import pandas as pd
import io
import contextlib

from langchain.agents.agent_types import AgentType
from langchain_experimental.agents.agent_toolkits import create_pandas_dataframe_agent

# Replace this with your actual LLM setup
# Example:
# from langchain_openai import AzureChatOpenAI
# model = AzureChatOpenAI(...)

# Prompt
CSV_PROMPT_PREFIX = """
Set pandas to show all columns.
Get the column names and infer data types.
Then attempt to answer the question using multiple methods.
Please provide only the Python code required to perform the action, and nothing else.
"""

CSV_PROMPT_SUFFIX = """
- Try at least 2 different methods of calculation or filtering.
- Reflect: Do they give the same result?
- After performing all necessary actions and analysis with the dataframe, return the answer in clean **Markdown**, include summary table if needed.
- Include **Execution Recommendation** and **Web Insight** in the final Markdown.
- Always conclude the final Markdown with:

### Final Answer

Your conclusion here.

---

### Explanation

Mention specific columns you used.
Please provide only the Python code required to perform the action, and nothing else until the final Markdown output.
"""

# --- Agent Logic ---
def ask_agent(files, question):
    try:
        dfs = [pd.read_csv(f.name) for f in files]
        df = pd.concat(dfs, ignore_index=True)
    except Exception as e:
        return f"❌ Could not read CSVs: {e}", ""

    try:
        agent = create_pandas_dataframe_agent(
        llm=model,
        df=df,
        verbose=True,
        agent_type=AgentType.ZERO_SHOT_REACT_DESCRIPTION,
        allow_dangerous_code=True,
        handle_parsing_errors=True,  # 👈 this is the fix
    )


        full_prompt = CSV_PROMPT_PREFIX + question + CSV_PROMPT_SUFFIX

        buffer = io.StringIO()
        with contextlib.redirect_stdout(buffer):
            result = agent.invoke(full_prompt)
        trace = buffer.getvalue()
        output = result["output"]


        return output, trace

    except Exception as e:
        return f"❌ Agent error: {e}", ""

# --- Gradio UI ---
with gr.Blocks(
    css="""
    body, .gradio-container {
        background: #ffffff !important;
        color: #1f2937 !important;
        font-family: 'Segoe UI', sans-serif;
    }

    #title {
        color: #1f2937 !important;
        font-size: 2rem;
        font-weight: 600;
        text-align: center;
        padding-top: 20px;
        padding-bottom: 10px;
    }

    .gr-box, .gr-input, .gr-output, .gr-markdown, .gr-textbox, .gr-file, textarea, input {
        background: rgba(0, 0, 0, 0.04) !important;
        border: 1px solid rgba(0, 0, 0, 0.1);
        border-radius: 12px !important;
        color: #1f2937 !important;
    }

    textarea::placeholder, input::placeholder {
        color: rgba(31, 41, 55, 0.6) !important;
    }

    button {
        background: rgba(0, 0, 0, 0.07) !important;
        color: #1f2937 !important;
        border: 1px solid rgba(0, 0, 0, 0.15) !important;
        border-radius: 8px !important;
    }

    button:hover {
        background: rgba(0, 0, 0, 0.15) !important;
    }
    """
) as demo:

    gr.Markdown("<h2 id='title'>📊 NexDatawork Data Agent</h2>")

    with gr.Column():
        result_display = gr.Markdown(label="📌 Report Output (Markdown)")
        trace_display = gr.Textbox(label="🛠️ Data Agent Reasoning - Your Explainable Agent", lines=20)

    with gr.Row(equal_height=True):
        file_input = gr.File(label="📁 Upload CSV(s)", file_types=[".csv"], file_count="multiple")
        question_input = gr.Textbox(
    label="💬 Ask Your Data",
    placeholder="e.g., What is the trend for revenue over time?",
    lines=9
)


    ask_button = gr.Button("💡 Analyze")

    ask_button.click(
        fn=ask_agent,
        inputs=[file_input, question_input],
        outputs=[result_display, trace_display]
    )

demo.launch(share=True)